# Library

* tensorflow_datasets is needed to download data from google cloud storage
* tensorflow_text is needed in the tokenizer library

In [ ]:
%pip install -q tensorflow_datasets
%pip install -q tensorflow_text tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 43.7 MB/s eta 0:00:00


In [ ]:
import pathlib
import tensorflow as tf
import tensorflow_text as tftxt
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import tensorflow_datasets as tfds
from pickle import dump, load
import numpy as np
from numpy.random import rand, shuffle
import re
from unicodedata import normalize
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/transformer_tf')
path = '/content/drive/MyDrive/Colab Notebooks/transformer_tf/deu-eng/metadata/'

In [ ]:
pathlib.Path.cwd()

PosixPath('/content')

# Data

## Process the data pairs

In [ ]:
filename = 'deu.txt'
with open(path + filename, 'rt', encoding='utf-8') as f:
  text = f.read()
  f.close()

In [ ]:
print(type(text))

<class 'str'>


In [ ]:
lines = text.strip().split('\n')
pairs = [l.split('\t')[0:2] for l in lines]
for i, p in enumerate(pairs):
  if i ==10: break
  print(p)

['Go.', 'Geh.']
['Hi.', 'Hallo!']
['Hi.', 'Grüß Gott!']
['Run!', 'Lauf!']
['Run.', 'Lauf!']
['Wow!', 'Potzdonner!']
['Wow!', 'Donnerwetter!']
['Duck!', 'Kopf runter!']
['Fire!', 'Feuer!']
['Help!', 'Hilfe!']


In [ ]:
cleaned = list()
for i, pair in enumerate(pairs):
  clean_pair = list()
  for sentence in pair:
    sentence = normalize('NFD', sentence)
    sentence = sentence.encode('utf-8')
    clean_pair.append(sentence)
  cleaned.append(clean_pair[::-1])

for i, p in enumerate(cleaned):
  if i ==10: break
  print(p)

[b'Geh.', b'Go.']
[b'Hallo!', b'Hi.']
[b'Gru\xcc\x88\xc3\x9f Gott!', b'Hi.']
[b'Lauf!', b'Run!']
[b'Lauf!', b'Run.']
[b'Potzdonner!', b'Wow!']
[b'Donnerwetter!', b'Wow!']
[b'Kopf runter!', b'Duck!']
[b'Feuer!', b'Fire!']
[b'Hilfe!', b'Help!']


In [ ]:
n_sentence = len(cleaned)
print(n_sentence)
train_ratio = 0.8
val_ratio   = 0.1
test_ratio  = 0.1
dataset = np.array(cleaned)
shuffle(dataset)

# dataset = cleaned[:10]

for i, p in enumerate(dataset):
  if i ==10: break
  print(p)

train = dataset[ : int(n_sentence*train_ratio)]
val = dataset[int(n_sentence*train_ratio) : int(n_sentence*(train_ratio+val_ratio))]
test = dataset[int(n_sentence*(train_ratio+val_ratio)) : int(n_sentence*(train_ratio+val_ratio+test_ratio))]

print(len(train), len(val), len(test))

261499
[b'Ich bekomme diesen Splitter nicht aus meinem Finger.'
 b"I can't get this splinter out of my finger."]
[b'Diese Entscheidung hat Tom getroffen.'
 b'Tom was the one who made that decision.']
[b'Tom wurde ermordet.' b'Tom has been murdered.']
[b'Tom sagte, er habe gewusst, dass ich ka\xcc\x88me.'
 b'Tom said he knew that I would come.']
[b'Tom ist nicht beru\xcc\x88hmt.' b'Tom is not famous.']
[b'Glaubst du das alles?' b'Do you believe all that?']
[b'Tom ist sich der Gefahr bewusst.' b'Tom is aware of the danger.']
[b'Wa\xcc\x88ren Sie bitte so nett, mir kurz Ihr Handy auszuleihen?'
 b'Would you be so kind as to let me borrow your cell phone, please?']
[b'Ihr mu\xcc\x88sst mehr u\xcc\x88ben!' b'You need to practice more.']
[b'Ich habe von meinem Vater ein Buch bekommen.'
 b'I got a book from my father.']
209199 26150 26150


## Save to txt

In [ ]:
%%time
with open(path + "deu-eng-train.txt", "w") as output:
  for i, p in enumerate(train):
    output.write(p[0].decode('utf-8') + '\t' + p[1].decode('utf-8') + '\n')

with open(path + "deu-eng-val.txt", "w") as output:
  for i, p in enumerate(val):
    output.write(p[0].decode('utf-8') + '\t' + p[1].decode('utf-8') + '\n')

with open(path + "deu-eng-test.txt", "w") as output:
  for i, p in enumerate(test):
    output.write(p[0].decode('utf-8') + '\t' + p[1].decode('utf-8') + '\n')

CPU times: user 912 ms, sys: 20.2 ms, total: 932 ms
Wall time: 1.16 s


## Save to Pkl

In [ ]:
%%time
dump(train, open(path + "deu-eng-train.pkl", 'wb'))
dump(val, open(path + "deu-eng-val.pkl", 'wb'))
dump(test, open(path + "deu-eng-test.pkl", 'wb'))

CPU times: user 66.8 ms, sys: 527 ms, total: 593 ms
Wall time: 2.12 s


## Load from Pkl

In [ ]:
filename = 'eng-deu-train.pkl'
with open(path + filename, 'rb') as file:
  dataset = load(file)
for i, pair in enumerate(dataset):
  if i == 3: break
  print(pair[0])
  print(pair[0].decode('utf-8'))
  # print(pair[0].decode('utf-8'))
  print(pair[1])
  # print(pair[1].decode('utf-8'))
  print('\n')

b'Wartest du kurz?'
Wartest du kurz?
b'Will you wait a second?'


b'Ich kann dir helfen, wenn du mich la\xcc\x88sst.'
Ich kann dir helfen, wenn du mich lässt.
b'I can help you if you let me.'


b'Wann hast du das letztemal eine Eule gesehen?'
Wann hast du das letztemal eine Eule gesehen?
b'When was the last time you saw an owl?'




In [ ]:
trainX = np.array(dataset)[:, 0]
trainY = np.array(dataset)[:, 1]
train_deu = tf.data.Dataset.from_tensor_slices(trainX)
train_eng = tf.data.Dataset.from_tensor_slices(trainY)


In [ ]:
train_deu

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [ ]:
for i, e in enumerate(train_deu):
  if i == 3: break
  print(e, type(e))

tf.Tensor(b'Wartest du kurz?', shape=(), dtype=string) <class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(b'Ich kann dir helfen, wenn du mich la\xcc\x88sst.', shape=(), dtype=string) <class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(b'Wann hast du das letztemal eine Eule gesehen?', shape=(), dtype=string) <class 'tensorflow.python.framework.ops.EagerTensor'>


# Create Vocabulary

In [ ]:
bert_tokenizer_params = dict(lower_case=True)
bert_tokenizer_params

{'lower_case': True}

In [ ]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]
reserved_tokens

['[PAD]', '[UNK]', '[START]', '[END]']

In [ ]:
bert_vocab_args = dict(
    vocab_size = 8000,
    reserved_tokens = reserved_tokens,
    bert_tokenizer_params = bert_tokenizer_params,
    learn_params = {}
)
bert_vocab_args

{'vocab_size': 8000,
 'reserved_tokens': ['[PAD]', '[UNK]', '[START]', '[END]'],
 'bert_tokenizer_params': {'lower_case': True},
 'learn_params': {}}

In [ ]:
%%time
deu_vocab = bert_vocab.bert_vocab_from_dataset(
    train_deu.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 3min 19s, sys: 398 ms, total: 3min 19s
Wall time: 3min 22s


In [ ]:
type(deu_vocab)

list

In [ ]:
print(deu_vocab[:20])
print(deu_vocab[100:120])
print(deu_vocab[1000:1020])
print(deu_vocab[-20:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '$', '%', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3']
['haben', 'an', 'sind', 'kann', 'noch', 'einen', 'so', 'bin', 'von', 'dem', 'fur', 'als', 'dich', 'hast', 'sehr', 'sein', 'dir', 'hier', 'hatte', 'wurde']
['mantel', 'acht', 'angekommen', 'betrunken', 'gitarre', 'schlaft', 'schreibtisch', 'ausland', 'freunden', 'gleiche', 'himmel', 'konne', 'passt', 'wert', 'ziehen', 'baby', 'eher', 'elke', 'fernseher', 'gespielt']
['##/', '##:', '##;', '##?', '##j', '##q', '##°', '##ˋ', '##а', '##–', '##—', '##‘', '##’', '##‚', '##“', '##”', '##„', '##‟', '##‽', '##⁄']


In [ ]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

write_vocab_file(path + 'deu_vocab.txt', deu_vocab)

In [ ]:
%%time
eng_vocab = bert_vocab.bert_vocab_from_dataset(
    train_eng.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 48.6 s, sys: 85.7 ms, total: 48.7 s
Wall time: 48.9 s


In [ ]:
print(eng_vocab[:10])
print(eng_vocab[100:110])
print(eng_vocab[1000:1010])
print(eng_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '$', '%', "'", '(']
['there', 'has', 'll', 've', 'here', 'very', 'go', 'think', 'about', 'didn']
['earth', 'keys', 'lend', 'patient', 'sign', 'voice', 'forward', 'wears', 'accept', 'notice']
['##?', '##j', '##q', '##°', '##–', '##—', '##’', '##“', '##”', '##€']


In [ ]:
write_vocab_file(path + 'eng_vocab.txt', eng_vocab)

# Build Tokensizer

In [ ]:
deu_tokenizer = tftxt.BertTokenizer(path + 'deu_vocab.txt', **bert_tokenizer_params)
eng_tokenizer = tftxt.BertTokenizer(path + 'eng_vocab.txt', **bert_tokenizer_params)

In [ ]:
# en_examples is 1 batch of size 3.
# it is a eagerTensor
print(trainX[:3])

print('\n')
token_batch = deu_tokenizer.tokenize(trainX[:3])
print(token_batch.shape)
print(token_batch)
print(token_batch.to_list())

print('\n')
token_batch = token_batch.merge_dims(-2,-1)
print(token_batch.shape)
print(token_batch)
print(token_batch.to_list())

print('\n')
for ex in token_batch.to_list():
  print(ex)


[b'Wartest du kurz?' b'Ich kann dir helfen, wenn du mich la\xcc\x88sst.'
 b'Wann hast du das letztemal eine Eule gesehen?']


(3, None, None)
<tf.RaggedTensor [[[2956], [74], [637], [28]],
 [[69], [103], [116], [213], [12], [130], [74], [95], [384], [14]],
 [[232], [113], [74], [73], [2215], [96], [33, 4005], [222], [28]]]>
[[[2956], [74], [637], [28]], [[69], [103], [116], [213], [12], [130], [74], [95], [384], [14]], [[232], [113], [74], [73], [2215], [96], [33, 4005], [222], [28]]]


(3, None)
<tf.RaggedTensor [[2956, 74, 637, 28], [69, 103, 116, 213, 12, 130, 74, 95, 384, 14],
 [232, 113, 74, 73, 2215, 96, 33, 4005, 222, 28]]>
[[2956, 74, 637, 28], [69, 103, 116, 213, 12, 130, 74, 95, 384, 14], [232, 113, 74, 73, 2215, 96, 33, 4005, 222, 28]]


[2956, 74, 637, 28]
[69, 103, 116, 213, 12, 130, 74, 95, 384, 14]
[232, 113, 74, 73, 2215, 96, 33, 4005, 222, 28]


In [ ]:
another_batch = [0, 1, 2, 3, 4, 5]
txt_tokens = tf.gather(deu_vocab, another_batch)
print(txt_tokens)

tf.Tensor([b'[PAD]' b'[UNK]' b'[START]' b'[END]' b'!' b'"'], shape=(6,), dtype=string)


In [ ]:
another_batch = [[1, 2, 3], [4, 5, 6], [7,8,9]]
txt_tokens = tf.gather(deu_vocab, another_batch)
print(type(txt_tokens))
print(txt_tokens)

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[b'[UNK]' b'[START]' b'[END]']
 [b'!' b'"' b'$']
 [b'%' b"'" b'(']], shape=(3, 3), dtype=string)


In [ ]:
txt_tokens = tf.gather(deu_vocab, token_batch)
print(txt_tokens)
tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)

<tf.RaggedTensor [[b'wartest', b'du', b'kurz', b'?'],
 [b'ich', b'kann', b'dir', b'helfen', b',', b'wenn', b'du', b'mich',
  b'lasst', b'.']                                                   ,
 [b'wann', b'hast', b'du', b'das', b'letztemal', b'eine', b'e', b'##ule',
  b'gesehen', b'?']                                                      ]>


<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'wartest du kurz ?',
       b'ich kann dir helfen , wenn du mich lasst .',
       b'wann hast du das letztemal eine e ##ule gesehen ?'], dtype=object)>

In [ ]:
words = deu_tokenizer.detokenize(token_batch)
print(words)

<tf.RaggedTensor [[b'wartest', b'du', b'kurz', b'?'],
 [b'ich', b'kann', b'dir', b'helfen', b',', b'wenn', b'du', b'mich',
  b'lasst', b'.']                                                   ,
 [b'wann', b'hast', b'du', b'das', b'letztemal', b'eine', b'eule',
  b'gesehen', b'?']                                               ]>


In [ ]:
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'wartest du kurz ?',
       b'ich kann dir helfen , wenn du mich lasst .',
       b'wann hast du das letztemal eine eule gesehen ?'], dtype=object)>

## ADD START, END

In [ ]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")
print(START, END)

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  print(count)
  starts = tf.fill([count, 1], START)
  ends = tf.fill([count, 1], END)
  return tf.concat([starts, ragged, ends], axis=1)

tf.Tensor(2, shape=(), dtype=int64) tf.Tensor(3, shape=(), dtype=int64)


In [ ]:
words = eng_tokenizer.detokenize(add_start_end(token_batch))
tf.strings.reduce_join(words, separator=' ', axis=-1)

tf.Tensor(3, shape=(), dtype=int64)


<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'[START] buses have song ? [END]',
       b'[START] he ve up asked , when have at early . [END]',
       b'[START] stay why have was nature she e babysitter leave ? [END]'],
      dtype=object)>

## Cleanup detokenized text

In [ ]:
bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
bad_tokens_re = "|".join(bad_tokens)
print(bad_tokens_re)
token_txt = ["[PAD]", "hello", "world", "[END]"]
bad_cells = tf.strings.regex_full_match(token_txt, bad_tokens_re)
print(bad_cells)
result = tf.ragged.boolean_mask(token_txt, ~bad_cells)
print(result)
result = tf.strings.reduce_join(result, separator=' ', axis=-1)
print(result)


\[PAD\]|\[START\]|\[END\]
tf.Tensor([ True False False  True], shape=(4,), dtype=bool)
tf.Tensor([b'hello' b'world'], shape=(2,), dtype=string)
tf.Tensor(b'hello world', shape=(), dtype=string)


In [ ]:
def cleanup_text(reserved_tokens, token_txt):
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_tokens_re = "|".join(bad_tokens)
  #
  bad_cells = tf.strings.regex_full_match(token_txt, bad_tokens_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)
  return result


In [ ]:
eng_examples = trainY[:3]
print(eng_examples)
print(eng_examples.dtype)
eng_examples = eng_examples.astype('str')
print(eng_examples)
print(eng_examples.dtype)

[b'Will you wait a second?' b'I can help you if you let me.'
 b'When was the last time you saw an owl?']
|S537
['Will you wait a second?' 'I can help you if you let me.'
 'When was the last time you saw an owl?']
<U537


In [ ]:
token_batch = eng_tokenizer.tokenize(eng_examples).merge_dims(-2,-1)
words = eng_tokenizer.detokenize(token_batch)
words

<tf.RaggedTensor [[b'will', b'you', b'wait', b'a', b'second', b'?'],
 [b'i', b'can', b'help', b'you', b'if', b'you', b'let', b'me', b'.'],
 [b'when', b'was', b'the', b'last', b'time', b'you', b'saw', b'an', b'owl',
  b'?']                                                                    ]>

In [ ]:
cleanup_text(reserved_tokens, words).numpy()

array([b'will you wait a second ?', b'i can help you if you let me .',
       b'when was the last time you saw an owl ?'], dtype=object)

# Export the Model

In [ ]:
vocab_path = path + 'deu_vocab.txt'
vocab = pathlib.Path(vocab_path).read_text().splitlines()
# tf.Variable(vocab)
print(type(vocab))
tf.Variable(vocab)

<class 'list'>


<tf.Variable 'Variable:0' shape=(7974,) dtype=string, numpy=
array([b'[PAD]', b'[UNK]', b'[START]', ..., b'##\xe2\x80\x9f',
       b'##\xe2\x80\xbd', b'##\xe2\x81\x84'], dtype=object)>

In [ ]:
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = tftxt.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)
    #
    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create signatures for export
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
        tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
        tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))
    ##
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  ##
  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    words = cleanup_text(self._reserved_tokens, words)
    return words

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)


In [ ]:
tokenizers = tf.Module()
tokenizers.deu = CustomTokenizer(reserved_tokens, path + 'deu_vocab.txt')
tokenizers.eng = CustomTokenizer(reserved_tokens, path + 'eng_vocab.txt')

Tensor("strided_slice:0", shape=(), dtype=int64)
Tensor("strided_slice:0", shape=(), dtype=int64)


In [ ]:
model_name = path + 'tokenizer_deu_eng_1'
tf.saved_model.save(tokenizers, model_name)

Tensor("strided_slice:0", shape=(), dtype=int64)
Tensor("strided_slice:0", shape=(), dtype=int64)


# Load and Test The Model

In [ ]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.deu.get_vocab_size().numpy()

7974

In [ ]:
tokens = reloaded_tokenizers.deu.tokenize(['Ich habe mein Geld für Kleidung, Essen und Bücher ausgegeben.'])
tokens.numpy()

array([[   2,   69,   86,  137,  194,  110, 1393,   12,  226,   97,  398,
        2685,   14,    3]])

In [ ]:
text_tokens = reloaded_tokenizers.deu.lookup(tokens)
text_tokens

<tf.RaggedTensor [[b'[START]', b'ich', b'habe', b'mein', b'geld', b'fur', b'kleidung',
  b',', b'essen', b'und', b'bucher', b'ausgegeben', b'.', b'[END]']]>

In [ ]:
round_trip = reloaded_tokenizers.deu.detokenize(tokens)
print(round_trip.numpy()[0].decode('utf-8'))

ich habe mein geld fur kleidung , essen und bucher ausgegeben .


In [ ]:
reloaded_tokenizers = tf.saved_model.load(path + 'tokenizer_deu_eng_1')
string = "When writing a sentence, generally you start with a capital letter and finish with a period (.), an exclamation mark (!), or a question mark (?)."
tokens = reloaded_tokenizers.eng.tokenize([string])
tokens.numpy()
text_tokens = reloaded_tokenizers.eng.lookup(tokens)
text_tokens
round_trip = reloaded_tokenizers.eng.detokenize(tokens)
print(round_trip.numpy()[0].decode('utf-8'))

when writing a sentence , generally you start with a capital letter and finish with a period ( . ) , an exclamation mark ( ! ) , or a question mark ( ? ) .


# Conclusion